In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
from answer_judge import load_judge

In [ ]:
data_reader=ReadData('/root/autodl-fs/hallscope/jailbreak')
data=data_reader.read_all()

random.shuffle(data)
# Set the seed np.random.seed(42) # Generate random numbers 
random_numbers = np.random.rand(5)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,is_safe,scenario,most_likely
0,[/home/hallscope/data/MM-SafetyBench/safe_data...,[Given the instruction 'The bottom of the imag...,"[tensor(1), tensor(1), tensor(1), tensor(1), t...","[09-Government.json, 09-Government.json, 09-Go...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,[/home/hallscope/data/MM-SafetyBench/data/imgs...,[Given the instruction 'The bottom of the imag...,"[tensor(0), tensor(0), tensor(0), tensor(0), t...","[09-Privacy_Violence.json, 09-Privacy_Violence...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,[/home/hallscope/data/MM-SafetyBench/data/imgs...,[Given the instruction 'The bottom of the imag...,"[tensor(0), tensor(0), tensor(0), tensor(0), t...","[08-Political_Lobbying.json, 08-Political_Lobb...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,[/home/hallscope/data/MM-SafetyBench/data/imgs...,[Given the instruction 'The bottom of the imag...,"[tensor(0), tensor(0), tensor(0), tensor(0), t...","[09-Privacy_Violence.json, 09-Privacy_Violence...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,[/home/hallscope/data/MM-SafetyBench/safe_data...,[Given the instruction 'What are the responsib...,"[tensor(1), tensor(1), tensor(1), tensor(1), t...","[09-Government.json, 09-Government.json, 09-Go...",{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(3573, 33, 4096)

In [6]:
gt_label=df['is_safe'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
print(f'There are {len(gt_label)} questions. {sum(gt_label==1)} answers are safe and {sum(gt_label==0)} answers are unsafe')

There are 3573 questions. 1800 answers are safe and 1773 answers are unsafe


In [7]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [8]:
resps[:5]

['Yes', 'Yes', 'Yes', 'Yes', 'Yes']

In [9]:
is_hall=[]
judge=load_judge(dataset='MMSafety',prompt_type='mq')
for i,x in enumerate(resps):
    is_hall.append(judge.check(x,gt_label[i]))
is_hall=np.array(is_hall)

In [10]:
print(f'There are {len(is_hall)} responses. {sum(is_hall==1)} answer correctly and {sum(is_hall==0)} answer wrongly')

There are 3573 responses. 1862 answer correctly and 1711 answer wrongly


In [ ]:
length=len(is_hall)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_hall[test_idxs]
hall_label_wild = is_hall[train_idxs]
hall_label_val = is_hall[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:1358
validset length:1500
testset length:715
trainset postive: 717 negative: 641
valset postive: 769 negative: 731
testset postive: 376 negative: 339


In [31]:
all_embeddings = embeddings
all_embeddings.shape

(3573, 33, 4096)

In [32]:
embedding_train = all_embeddings[train_idxs][:, 1:, :]
embedding_val = all_embeddings[val_idxs][:, 1:, :]
embedding_test = all_embeddings[test_idxs][:, 1:, :]

In [33]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [34]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [35]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.48381805923445975
found best n_components:1,layer:1 Score: 0.5527262830011794
found best n_components:1,layer:2 Score: 0.5934101707940563
found best n_components:1,layer:3 Score: 0.6052524375643746
found best n_components:1,layer:4 Score: 0.6347415852662774
found best n_components:1,layer:5 Score: 0.6413413764211342
found best n_components:1,layer:6 Score: 0.7036088938856759
found best n_components:1,layer:7 Score: 0.7487649851727064
found best n_components:1,layer:8 Score: 0.7934745676781009
found best n_components:1,layer:20 Score: 0.7996794387153355
found best n_components:1,layer:22 Score: 0.8067613170408032
found best n_components:1,layer:25 Score: 0.8241128973438954
found best n_components:1,layer:26 Score: 0.8242641054970391


100%|██████████| 11/11 [00:26<00:00,  2.43s/it]


In [36]:
best_paras

{'n_components': 1, 'layer': 26, 'best_score': np.float64(0.8242641054970391)}

In [37]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [38]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [39]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.7430044182621502 AUROC:0.797039580327983


In [40]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.634965034965035 AUROC:0.8015675014121635


Train Linear Prob to Get Better Results

In [44]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [45]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [46]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.47412587412587415 AUROC:0.49601063829787234
